In [1]:
import socket
import threading

class MessagingServer:
    def __init__(self): # Initialize the server
        self.name = socket.gethostname()
        
        self.ip_address = socket.gethostbyname(self.name)
        self.port = 317

        self.socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

        self.socket.bind((self.ip_address, self.port))


    def handle_client(self, client_conn, client_addr):
        
        while True:
            packet = client_conn.recv(1024).decode('utf-8').split('to')
            message, forward_user = packet[0], packet[1]

            print(message, forward_user)

            print(f"client ({client_addr} connected)")


    def startUp(self):
        self.socket.listen(5)
        count = 0
        
        while True:
            
            count += 1
            print(count)
            print(f"Active clients: {threading.activeCount() - 1}")
            
            client_conn, client_addr = self.socket.accept()
            
            thread = threading.Thread(target=self.handle_client, args = (client_conn, client_addr))
            
            thread.start()
            thread.join()
            
            # print(f"Active clients: {threading.activeCount() - 1}")

    def start(self):
        self.socket.listen(5)
        client_conn, client_addr = self.socket.accept()
        
        message = client_conn.recv(1024).decode('utf-8')
        
        print(message)
        
        client_conn.close()
        self.socket.close()


In [2]:
server = MessagingServer()

In [3]:
print(server.ip_address)
print(server.port)

192.168.10.4
317


In [ ]:
server.startUp()

1
Active clients: 4
from c1 c2
client (('192.168.10.4', 50513) connected)


In [7]:
del server

In [4]:
# def on_new_client(client_socket):
#     #while True:
#     msg = client_socket.recv(1024).decode('utf-8')
#     #do some checks and if msg == someWeirdSignal: break:
#     print (client_socket.client_addr, ' >> ', msg)
    
#     #Maybe some code to compute the last digit of PI, play game or anything else can go here and when you are done.
#     #clientsocket.send(msg)

#     clientsocket.close()

In [4]:
server.socket.listen(5)
c, addr = server.socket.accept()

In [ ]:
server.socket.listen(5)
c, addr = server.socket.accept()

for i in range(10): # while True:
         # c, addr = server.socket.accept() Establish connection with client.
    
    print(addr)
    
    packet = c.recv(1024).decode('utf-8').split('to')
    message, forward_user = packet[0], packet[1]
    
    print(message, forward_user)
    
#server.close()

('192.168.10.4', 57883)
from c1 c2
('192.168.10.4', 57883)
hau c2
('192.168.10.4', 57883)


In [3]:
server.socket.close()

NameError: name 'server' is not defined